### **NOTE: You Need GPU For this. I ran this notebook in Google Colab with T4 GPU Instance**
Colab Notebook Link: https://colab.research.google.com/drive/1Yg8f3OG98yfKhLa6zoXgF0xiobzVTkgT?usp=sharing (View Only)

# Imports and Installs

In [1]:
!pip3 uninstall transformers
!pip3 install transformers==4.26.0
!pip3 install trial2vec

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.44.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.44.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from trial2vec import Trial2Vec
from trial2vec import download_embedding

import torch
import torch.nn.functional as F
import json
import matplotlib.pyplot as plt
import seaborn as sns

t2v_emb = download_embedding()

model = Trial2Vec()
model.from_pretrained('./trial_search/pretrained_trial2vec')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Download pretrained Trial2Vec model, save to ./trial_search/pretrained_trial2vec.
Load pretrained Trial2Vec model from ./trial_search/pretrained_trial2vec


/usr/local/lib/python3.10/dist-packages/trial2vec/model.py:836: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_filename, map_location=torch

load predictor config file from ./trial_search/pretrained_trial2vec/model_config.json


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load pretrained Trial2Vec model from ./trial_search/pretrained_trial2vec
load predictor config file from ./trial_search/pretrained_trial2vec/model_config.json


In [3]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn.functional as F
import json
import random

# Helper Functions

In [4]:
def get_match_json(similarity_matrix, reference_features, candidate_features, threshold):

    # Initialize lists to keep track of matches and unmatched features
    matched_features = []
    remaining_reference_features = reference_features.copy()
    remaining_candidate_features = candidate_features.copy()

    iteration = 1
    while similarity_matrix.size(0) > 0 and similarity_matrix.size(1) > 0:

        # Find the maximum similarity value
        max_similarity, max_idx = torch.max(similarity_matrix, dim=1)
        max_value, ref_idx = torch.max(max_similarity, dim=0)
        max_value = max_value.item()
        ref_idx = ref_idx.item()

        if max_value < threshold or len(remaining_candidate_features) == 0:
            break

        # Find all pairs with the same maximum similarity value
        candidates = [(i, max_idx[i].item()) for i in range(similarity_matrix.size(0)) if max_similarity[i].item() == max_value]

        # Select one pair randomly
        ref_idx, cand_idx = random.choice(candidates)

        # Add the matched pair to the list of matched features
        matched_pair = (remaining_reference_features[ref_idx], remaining_candidate_features[cand_idx])
        matched_features.append(matched_pair)

        # Remove the matched pair from the lists
        del remaining_reference_features[ref_idx]
        del remaining_candidate_features[cand_idx]

        # Remove the corresponding row and column from the similarity matrix
        similarity_matrix = torch.cat((similarity_matrix[:ref_idx], similarity_matrix[ref_idx+1:]), dim=0)
        similarity_matrix = torch.cat((similarity_matrix[:, :cand_idx], similarity_matrix[:, cand_idx+1:]), dim=1)

        iteration += 1

    # Create the result dictionary
    result = {
        "matched_features": matched_features,
        "remaining_reference_features": remaining_reference_features,
        "remaining_candidate_features": remaining_candidate_features
    }

    return result

def extract_elements_v2(s):
  """
  Extracts elements enclosed within backticks (`) from a string.

  Args:
    s: The input string.

  Returns:
    A list of elements enclosed within backticks.
  """
  pattern = r"`(.*?)`"
  elements = re.findall(pattern, s)
  return elements


def plot_similarity_matrix(similarity_matrix_np, reference, candidate):

    # Plotting the similarity matrix using seaborn's heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(similarity_matrix_np, annot=True, cmap='coolwarm', fmt=".2f",
                     xticklabels=candidate, yticklabels=reference)
    plt.title("Pairwise Cosine Similarities")
    plt.xlabel("Candidate Components")
    plt.ylabel("Reference Components")
    plt.show()

def calculate_plot_and_return_matches(model, ref, can, threshold, show_plot=False):

    reference = extract_elements_v2(ref)
    candidate = extract_elements_v2(can)

    # Assume model.sentence_vector is a method that returns a tensor of shape [n, 128] for n sentences
    reference_tensor = model.sentence_vector(reference).cpu()
    candidate_tensor = model.sentence_vector(candidate).cpu()

    # Normalize the tensors
    reference_tensor = F.normalize(reference_tensor, p=2, dim=1)
    candidate_tensor = F.normalize(candidate_tensor, p=2, dim=1)

    # Compute the similarity matrix
    similarity_matrix = torch.mm(reference_tensor, candidate_tensor.transpose(0, 1))

    result = get_match_json(similarity_matrix, reference, candidate, threshold)
    if show_plot:
      plot_similarity_matrix(similarity_matrix.numpy(), reference, candidate)

    return json.dumps(result, indent=2), similarity_matrix

###### For saving similarity matrix in db #######
def encode_matrix(similarity_matrix):
    import pickle, base64
    similarity_matrix_serialized = pickle.dumps(similarity_matrix)
    similarity_matrix_encoded = base64.b64encode(similarity_matrix_serialized).decode('utf-8')
    return similarity_matrix_encoded

def decode_matrix(similarity_matrix_encoded):
    import pickle, base64
    similarity_matrix_serialized = base64.b64decode(similarity_matrix_encoded)
    similarity_matrix = pickle.loads(similarity_matrix_serialized)
    return similarity_matrix
###################################################

def run_bert_score(gen_model_name, model, doc_id, reference_list, candidate_list, bert_scores):

    result_07, _ = calculate_plot_and_return_matches(model, reference_list, candidate_list, threshold=0.7)

    return result_07

def match_to_score(matched_pairs, remaining_reference_features, remaining_candidate_features):
    """
    Calculates precision, recall, and F1 score based on the given matched pairs and remaining features.

    Parameters:
    matched_pairs (list): A list of matched feature pairs.
    remaining_reference_features (list): A list of remaining reference features.
    remaining_candidate_features (list): A list of remaining candidate features.

    Returns:
    dict: A dictionary containing the precision, recall, and F1 score.
    """
    precision = len(matched_pairs) / (len(matched_pairs) + len(remaining_candidate_features)) # TP/(TP+FP)
    recall = len(matched_pairs) /  (len(matched_pairs) + len(remaining_reference_features)) #TP/(TP+FN)

    if precision == 0 or recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall) # F1

    return {"precision": precision, "recall": recall, "f1": f1}

# Load Data with generated matches from all 4 models and the reference (CT-PUB)

In [5]:
import pandas as pd
data = pd.read_csv('CT-Repo-With-Examples-Corrected-allgen.csv')

In [6]:
data.head(3)

,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,TrialGroup,API_BaselineMeasures,API_BaselineMeasures_Corrected,gpt4o_zs_gen,gpt4o_ts_gen,llama3_70b_it_zs_gen,llama3_70b_it_ts_gen
0,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Inclusion Criteria:\n\n* Diagnosed with type 2...,The purpose of this study is to prevent major ...,"Atherosclerosis, Cardiovascular Diseases, Hype...","Anti-hyperglycemic Agents, Anti-hypertensive A...",First Occurrence of a Major Cardiovascular Eve...,hypertension,"Age, Continuous, Gender, Ethnicity (NIH/OMB), ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","`Age`, `Sex`, `Race/Ethnicity`, `Body Mass Ind...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","`Age`, `Sex`, `Race`, `Ethnicity`, `Body Mass ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O..."
1,NCT00003901,Prognostic Study of Metastases in Patients Wit...,Inclusion Criteria:\n\n1. Patient must be ≥ 18...,RATIONALE: Prognostic testing for early signs ...,"Lung Cancer,","immunohistochemistry staining method, biopsy, ...",Overall Survival in Lymph Nodes Examined Patie...,cancer,"Age, Continuous, Gender, Race/Ethnicity, Custo...","`Age, Continuous`, `Gender`, `Race/Ethnicity, ...","`Age`, `Gender`, `ECOG/Zubrod status`, `Clinic...","`Age, Continuous`, `Sex: Female, Male`, `Race/...","`age`, `sex`, `ECOG/Zubrod status`, `histologi...","`Age, Continuous`, `Sex: Female, Male`, `Race/..."
2,NCT00005879,LY353381 in Preventing Breast Cancer in Women ...,DISEASE CHARACTERISTICS:\n\n* Current random f...,RATIONALE: Chemoprevention therapy is the use ...,"Breast Cancer,","arzoxifene, Placebo,","Change in Masood Score, Number of Participants...",cancer,"Age, Continuous, Sex: Female, Male, Region of ...","`Age, Continuous`, `Sex: Female, Male`, `Regio...","`Age`, `Sex`, `Menopausal status`, `Hemoglobin...","`Age, Continuous`, `Sex: Female`, `Race/Ethnic...","`Age`, `Sex`, `Menopausal status`, `Body Mass ...","`Age, Continuous`, `Sex: Female`, `Race/Ethnic..."


## Generate Matches based on BERT Score and "ListMatch BERT" algorithm mentioned in Paper Supplement.

In [7]:
from tqdm import tqdm
from time import sleep

#bert_scores = pd.DataFrame(columns=['NCTId', 'Model', 'bert-score-matches-07'])
bert_scores = pd.DataFrame()
bert_scores['NCTId'] = data['NCTId']

threeshot_example_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783']

for index, row in tqdm(data.iterrows()):

    doc_id = row['NCTId']
    reference = row['API_BaselineMeasures_Corrected']

    print(f"{index+1}. Trial ID:{row['NCTId']}")

    if doc_id in threeshot_example_ids:
        continue

    #run_bert_score(gen_model_name, model, doc_id, reference_list, candidate_list, bert_scores):

    #eval gpt4-omni-zs
    candidate = row['gpt4o_zs_gen']
    bert_matches = run_bert_score('gpt4-omni-zs', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'gpt4o_zs_gen_matches'] = bert_matches

    #eval gpt4-omni-ts
    candidate = row['gpt4o_ts_gen']
    bert_matches = run_bert_score('gpt4-omni-ts', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'gpt4o_ts_gen_matches'] = bert_matches

    #eval llamma3-70b-in-zs
    candidate = row['llama3_70b_it_zs_gen']
    bert_matches = run_bert_score('llama3-70b-in-zs', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'llama3_70b_it_zs_gen_matches'] = bert_matches

    #eval llamma3-70b-in-ts
    candidate = row['llama3_70b_it_ts_gen']
    bert_matches = run_bert_score('llama3-70b-in-ts', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'llama3_70b_it_ts_gen_matches'] = bert_matches

    #break

#save the matches
bert_scores.to_csv('CT-Repo-With-Examples-Corrected-allbert07eval.csv', index=False)



0it [00:00, ?it/s]

1. Trial ID:NCT00000620
2. Trial ID:NCT00003901


2it [00:02,  1.42s/it]

3. Trial ID:NCT00005879


4it [00:03,  1.52it/s]

...

1689. Trial ID:NCT05204134
1690. Trial ID:NCT05289869


1691it [04:22,  7.55it/s]

1691. Trial ID:NCT05387889
1692. Trial ID:NCT05451329


1693it [04:23,  6.44it/s]


1693. Trial ID:NCT05741489


In [8]:
#matched document contains match, reference mismatch, candidate mismatch
bert_scores.head(5)

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00003901,"{\n ""matched_features"": [\n [\n ""Gend...","{\n ""matched_features"": [\n [\n ""Regi...","{\n ""matched_features"": [\n [\n ""Gend...","{\n ""matched_features"": [\n [\n ""Tumo..."
2,NCT00005879,"{\n ""matched_features"": [\n [\n ""Meno...","{\n ""matched_features"": [\n [\n ""Regi...","{\n ""matched_features"": [\n [\n ""Meno...","{\n ""matched_features"": [\n [\n ""Regi..."
3,NCT00005908,"{\n ""matched_features"": [\n [\n ""Age ...","{\n ""matched_features"": [\n [\n ""Regi...","{\n ""matched_features"": [\n [\n ""Age ...","{\n ""matched_features"": [\n [\n ""Regi..."
4,NCT00006110,"{\n ""matched_features"": [\n [\n ""Sex:...","{\n ""matched_features"": [\n [\n ""Regi...","{\n ""matched_features"": [\n [\n ""Sex:...","{\n ""matched_features"": [\n [\n ""Age,..."


## Calculate Bert Scores from BERT Matches

In [10]:
score_df = pd.DataFrame()
score_df['NCTId'] = data['NCTId']

In [11]:
import json

gzs_sum = {'precision':0, 'recall':0, 'f1':0}
gts_sum = {'precision':0, 'recall':0, 'f1':0}
lzs_sum = {'precision':0, 'recall':0, 'f1':0}
lts_sum = {'precision':0, 'recall':0, 'f1':0}

for index, row in bert_scores.iterrows():
    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    #gzs
    gzs_dict = json.loads(row['gpt4o_zs_gen_matches'])
    gzs_matches, gzs_remaining_references, gzs_remaining_candidates = gzs_dict['matched_features'], gzs_dict['remaining_reference_features'], gzs_dict['remaining_candidate_features']
    gzs_score = match_to_score(gzs_matches, gzs_remaining_references, gzs_remaining_candidates)
    score_df.at[index, 'gpt4o_zs_gen_scores'] = json.dumps(gzs_score)
    gzs_sum['precision'] += gzs_score['precision']
    gzs_sum['recall'] += gzs_score['recall']
    gzs_sum['f1'] += gzs_score['f1']

    #gts
    gts_dict = json.loads(row['gpt4o_ts_gen_matches'])
    gts_matches, gts_remaining_references, gts_remaining_candidates = gts_dict['matched_features'], gts_dict['remaining_reference_features'], gts_dict['remaining_candidate_features']
    gts_score = match_to_score(gts_matches, gts_remaining_references, gts_remaining_candidates)
    score_df.at[index, 'gpt4o_ts_gen_scores'] = json.dumps(gts_score)
    gts_sum['precision'] += gts_score['precision']
    gts_sum['recall'] += gts_score['recall']
    gts_sum['f1'] += gts_score['f1']

    #lzs
    lzs_dict = json.loads(row['llama3_70b_it_zs_gen_matches'])
    lzs_matches, lzs_remaining_references, lzs_remaining_candidates = lzs_dict['matched_features'], lzs_dict['remaining_reference_features'], lzs_dict['remaining_candidate_features']
    lzs_score = match_to_score(lzs_matches, lzs_remaining_references, lzs_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_zs_gen_scores'] = json.dumps(lzs_score)
    lzs_sum['precision'] += lzs_score['precision']
    lzs_sum['recall'] += lzs_score['recall']
    lzs_sum['f1'] += lzs_score['f1']

    #lts
    lts_dict = json.loads(row['llama3_70b_it_ts_gen_matches'])
    lts_matches, lts_remaining_references, lts_remaining_candidates = lts_dict['matched_features'], lts_dict['remaining_reference_features'], lts_dict['remaining_candidate_features']
    lts_score = match_to_score(lts_matches, lts_remaining_references, lts_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_ts_gen_scores'] = json.dumps(lts_score)
    lts_sum['precision'] += lts_score['precision']
    lts_sum['recall'] += lts_score['recall']
    lts_sum['f1'] += lts_score['f1']

    #break



In [12]:
score_df.head()

,NCTId,gpt4o_zs_gen_scores,gpt4o_ts_gen_scores,llama3_70b_it_zs_gen_scores,llama3_70b_it_ts_gen_scores
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00003901,"{""precision"": 0.1111111111111111, ""recall"": 0....","{""precision"": 0.3333333333333333, ""recall"": 0....","{""precision"": 0.18181818181818182, ""recall"": 0...","{""precision"": 0.45454545454545453, ""recall"": 0..."
2,NCT00005879,"{""precision"": 0.0625, ""recall"": 0.166666666666...","{""precision"": 0.2857142857142857, ""recall"": 0....","{""precision"": 0.2, ""recall"": 0.333333333333333...","{""precision"": 0.2608695652173913, ""recall"": 0...."
3,NCT00005908,"{""precision"": 0.0625, ""recall"": 0.166666666666...","{""precision"": 0.2857142857142857, ""recall"": 0....","{""precision"": 0.11764705882352941, ""recall"": 0...","{""precision"": 0.2857142857142857, ""recall"": 0...."
4,NCT00006110,"{""precision"": 0.05263157894736842, ""recall"": 0...","{""precision"": 0.2, ""recall"": 0.428571428571428...","{""precision"": 0.0625, ""recall"": 0.142857142857...","{""precision"": 0.06666666666666667, ""recall"": 0..."


In [13]:
#save bert scores
score_df.to_csv('CT-Repo-With-Examples-Corrected-bertmetric.csv', index=False)

In [15]:
score_df.shape

(1693, 5)

## Final Scores

In [16]:
size = 1690
print(f"GPT-4o Zero Shot Scores: Precision={gzs_sum['precision']/size} Recall={gzs_sum['recall']/size} F1={gzs_sum['f1']/size}")
print(f"GPT-4o Three Shot Scores: Precision={gts_sum['precision']/size} Recall={gts_sum['recall']/size} F1={gts_sum['f1']/size}")
print(f"Llama-3 70B Instruct Zero Shot Scores: Precision={lzs_sum['precision']/size} Recall={lzs_sum['recall']/size} F1={lzs_sum['f1']/size}")
print(f"Llama-3 70B Instruct Three Shot Scores: Precision={lts_sum['precision']/size} Recall={lts_sum['recall']/size} F1={lts_sum['f1']/size}")

GPT-4o Zero Shot Scores: Precision=0.11919943981446339 Recall=0.20557350568423557 F1=0.14233219539428518
GPT-4o Three Shot Scores: Precision=0.35203203711574327 Recall=0.5001452994192461 F1=0.39530252174719627
Llama-3 70B Instruct Zero Shot Scores: Precision=0.1602543616840975 Recall=0.26408971624554145 F1=0.19040325807136668
Llama-3 70B Instruct Three Shot Scores: Precision=0.355274892568452 Recall=0.4901739025537531 F1=0.3915967265007171
